# Transformer Encoder

## Transformer in a nutshell

Transformer 由 Encoder 和 Decoder 两部分组成，Encoder 接受 **输入** 然後生成 **向量** 表示這個输入，Decoder 接受 **向量** 然後逐一生成 **输出**。
![transformer in a nutshell](https://www.alexisalulema.com/wp-content/uploads/2022/08/encoder-decoder-1-1024x275.png)

### Encoder

更詳細地說明時，Transformer 由 N 個 Encoder 組成，每個 Encoder 將其輸出送到下一個 Encoder。最終的 Encoder 返回 **輸入** 的 **向量** 表示。為了說明，從現在開始，我們將使用 N=2 的值。

![encoder in a nutshell](https://www.alexisalulema.com/wp-content/uploads/2022/08/encoders.png)

### Decoder

同樣地，我們可以有 N 個 Decoder（假設 N=2）。Encoder 生成的 **向量** 表示是所有 Decoder 的輸入；也就是說，一個 Decoder 接收兩個輸入，一個來自前一個 Decoder，另一個來自 Encoder 生成的 **向量** 表示。

![decoder in a nutshell](https://www.alexisalulema.com/wp-content/uploads/2022/08/encoderNdecoder.png)